# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [7]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [8]:
log = True

In [9]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    word_counts = Counter(words)
    # sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    # create int_to_vocab dictionaries
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [10]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    punctuation_lookup = {
        '.': '<PERIOD>',
        ',': '<COMMA>',
        '"': '<QUOTATION_MARK>',
        ';': '<SEMICOLON>',
        '!': '<EXCLAMATION_MARK>',
        '?': '<QUESTION_MARK>',
        '(': '<LEFT_PAREN>',
        ')': '<RIGHT_PAREN>',
        '-': '<DASH>',
        '\n': '<NEW_LINE>',
    }
    return punctuation_lookup
        
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [14]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
     Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: data_loaders: DataLoader with batched data
    """
    n_batches = len(words) // batch_size
    # Filter the residual data so that we have only full atches
    words = words[:n_batches * batch_size]
    future_tensors, targets = [], []
    for ii in range(0, len(words) - sequence_length):
        start = ii
        end = sequence_length + ii
        data = words[start:end]
        future_tensors.append(data)
        if end  >= (len(words)):
            target_word = words[0]
        else:
            target_word = words[end]
        targets.append(target_word)
    # create Tensor datasets
    data = TensorDataset(torch.from_numpy(np.asarray(future_tensors)), torch.from_numpy(np.asarray(targets)))
    data_loader = DataLoader(data, shuffle=False, batch_size=batch_size)
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [15]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)
data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()
print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 0,  1,  2,  3,  4],
        [ 1,  2,  3,  4,  5],
        [ 2,  3,  4,  5,  6],
        [ 3,  4,  5,  6,  7],
        [ 4,  5,  6,  7,  8],
        [ 5,  6,  7,  8,  9],
        [ 6,  7,  8,  9, 10],
        [ 7,  8,  9, 10, 11],
        [ 8,  9, 10, 11, 12],
        [ 9, 10, 11, 12, 13]], dtype=torch.int32)

torch.Size([10])
tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14], dtype=torch.int32)


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [16]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, output_size, embed_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embed_dim: The size of embeddings, should you choose to use them
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # Internal Variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        # Layers
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, n_layers,
                            dropout=dropout, batch_first=True)
        # dropout layer
        self.dropout = nn.Dropout(dropout)
        # linear  layers
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # nn_input = nn_input.long()
        nn_input = nn_input.to(torch.int64)
        batch_size = nn_input.size(0)
        # embeddings and lstm_out
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # reshape into (batch_size, seq_length, output_size)
        out = out.view(batch_size, -1, self.output_size)
        out = out[:, -1]  # get last batch of labels
        # return one batch of output word scores and the hidden state
        return out, hidden

    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if  train_on_gpu:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

        return hidden
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [17]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
     Forward and backward propagation on the neural network
    :param train_on_gpu: Flag to indicate of GPU is available
    :param hidden: Hidden state output
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """

    # TODO: Implement Function

    # move data to GPU, if available

    # perform backpropagation and optimization

    # return the loss over a batch and the hidden state produced by our model
    gradient_clip = 5  # gradient clipping

    # Creating new variables for the hidden state
    hidden = tuple([each.data for each in hidden])
    # move data to GPU, if available
    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()
    # zero accumulated gradients
    rnn.zero_grad()
    # perform backpropagation and optimization
    output, hidden = rnn(inp, hidden)
    loss = criterion(output, target)
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(), gradient_clip)
    optimizer.step()
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden


# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []
        helper.save_model('./save/trained_rnn_epoch', rnn)
        print('Model Trained and Saved')

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [21]:
# Data params
# Sequence Length
sequence_length =   32# of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)


In [22]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimension
embedding_dim = 200
# Hidden Dimension
hidden_dim = 1024
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 25

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.3)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 7.008833694458008

Epoch:    1/10    Loss: 6.115194435119629

Epoch:    1/10    Loss: 5.7657509040832515

Epoch:    1/10    Loss: 5.8467887496948245

Epoch:    1/10    Loss: 5.445368461608886

Epoch:    1/10    Loss: 5.442215938568115

Epoch:    1/10    Loss: 5.400918922424316

Epoch:    1/10    Loss: 5.244353694915771

Epoch:    1/10    Loss: 5.305827827453613

Epoch:    1/10    Loss: 5.006659746170044

Epoch:    1/10    Loss: 5.193786563873291

Epoch:    1/10    Loss: 5.053718776702881

Epoch:    1/10    Loss: 5.145954504013061

Epoch:    1/10    Loss: 4.766558475494385

Epoch:    1/10    Loss: 5.032445430755615

Epoch:    1/10    Loss: 5.0633285617828365

Epoch:    1/10    Loss: 4.790766859054566

Epoch:    1/10    Loss: 5.184553775787354

Epoch:    1/10    Loss: 5.17572865486145

Epoch:    1/10    Loss: 5.226237907409668

Epoch:    1/10    Loss: 4.866916236877441

Epoch:    1/10    Loss: 4.85487138748169

Epoch:    1/10    Loss: 5

Epoch:    1/10    Loss: 4.230617361068726

Epoch:    1/10    Loss: 3.9056504821777343

Epoch:    1/10    Loss: 4.103692359924317

Epoch:    1/10    Loss: 4.214705171585083

Epoch:    1/10    Loss: 4.421868257522583

Epoch:    1/10    Loss: 4.150262098312378

Epoch:    1/10    Loss: 4.324673147201538

Epoch:    1/10    Loss: 4.102217359542847

Epoch:    1/10    Loss: 3.992949752807617

Epoch:    1/10    Loss: 4.17227689743042

Epoch:    1/10    Loss: 4.257792072296143

Epoch:    1/10    Loss: 4.260800228118897

Epoch:    1/10    Loss: 4.5048912525177

Epoch:    1/10    Loss: 4.404163866043091

Epoch:    1/10    Loss: 4.331863222122192

Epoch:    1/10    Loss: 4.410363082885742

Epoch:    1/10    Loss: 4.206347932815552

Epoch:    1/10    Loss: 3.936979236602783

Epoch:    1/10    Loss: 4.288059682846069

Epoch:    1/10    Loss: 4.259829654693603

Epoch:    1/10    Loss: 4.224406127929687

Epoch:    1/10    Loss: 4.5217545700073245

Epoch:    1/10    Loss: 4.344159231185913

Epoch:    1/

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved
Epoch:    2/10    Loss: 4.4743899689164275

Epoch:    2/10    Loss: 4.137075719833374

Epoch:    2/10    Loss: 3.8318759632110595

Epoch:    2/10    Loss: 4.120935125350952

Epoch:    2/10    Loss: 3.851857290267944

Epoch:    2/10    Loss: 3.866565885543823

Epoch:    2/10    Loss: 4.041164169311523

Epoch:    2/10    Loss: 3.9318419170379637

Epoch:    2/10    Loss: 4.054263525009155

Epoch:    2/10    Loss: 3.844706163406372

Epoch:    2/10    Loss: 4.099021558761597

Epoch:    2/10    Loss: 3.950495195388794

Epoch:    2/10    Loss: 4.03415813446045

Epoch:    2/10    Loss: 3.7052436447143555

Epoch:    2/10    Loss: 4.054402408599853

Epoch:    2/10    Loss: 4.061418142318725

Epoch:    2/10    Loss: 3.7217216014862062

Epoch:    2/10    Loss: 4.090556306838989

Epoch:    2/10    Loss: 4.026103382110596

Epoch:    2/10    Loss: 4.146270198822021

Epoch:    2/10    Loss: 3.8799396324157716

Epoch:    2/10    Loss: 3.954551305770874

Epoch:    2/10    Loss: 4

Epoch:    2/10    Loss: 3.726663017272949

Epoch:    2/10    Loss: 3.835032625198364

Epoch:    2/10    Loss: 3.470537347793579

Epoch:    2/10    Loss: 3.694989252090454

Epoch:    2/10    Loss: 3.8016589069366455

Epoch:    2/10    Loss: 4.018163890838623

Epoch:    2/10    Loss: 3.774096841812134

Epoch:    2/10    Loss: 3.9162925815582277

Epoch:    2/10    Loss: 3.68971284866333

Epoch:    2/10    Loss: 3.5245525646209717

Epoch:    2/10    Loss: 3.794970817565918

Epoch:    2/10    Loss: 3.8648329734802247

Epoch:    2/10    Loss: 3.8787668132781983

Epoch:    2/10    Loss: 4.068665170669556

Epoch:    2/10    Loss: 3.9782222652435304

Epoch:    2/10    Loss: 3.928909034729004

Epoch:    2/10    Loss: 4.006608371734619

Epoch:    2/10    Loss: 3.909468870162964

Epoch:    2/10    Loss: 3.6532833576202393

Epoch:    2/10    Loss: 3.8845678997039794

Epoch:    2/10    Loss: 3.8797965621948243

Epoch:    2/10    Loss: 3.85100923538208

Epoch:    2/10    Loss: 4.137432708740234

Epoc

...

Epoch:    8/10    Loss: 3.116751823425293

Epoch:    8/10    Loss: 2.895250234603882

Epoch:    8/10    Loss: 3.1284564208984373

Epoch:    8/10    Loss: 2.993426790237427

Epoch:    8/10    Loss: 3.3826612281799315

Epoch:    8/10    Loss: 3.1601303386688233

Epoch:    8/10    Loss: 3.077616014480591

Epoch:    8/10    Loss: 2.921684808731079

Epoch:    8/10    Loss: 2.9669510078430177

Epoch:    8/10    Loss: 2.89727126121521

Epoch:    8/10    Loss: 2.973122825622559

Epoch:    8/10    Loss: 2.992047805786133

Epoch:    8/10    Loss: 2.8293513298034667

Epoch:    8/10    Loss: 2.8415130138397218

Epoch:    8/10    Loss: 3.0271427631378174

Epoch:    8/10    Loss: 2.9828167533874512

Epoch:    8/10    Loss: 3.345701036453247

Epoch:    8/10    Loss: 3.088503532409668

Epoch:    8/10    Loss: 2.974940881729126

Epoch:    8/10    Loss: 3.09303503036499

Epoch:    8/10    Loss: 2.9312508296966553

Epoch:    8/10    Loss: 3.099103422164917

Epoch:    8/10    Loss: 3.055289554595947

Epoc

Epoch:    8/10    Loss: 3.0327204990386964

Epoch:    8/10    Loss: 2.8934968280792237

Epoch:    8/10    Loss: 3.072907361984253

Epoch:    8/10    Loss: 3.053640432357788

Epoch:    8/10    Loss: 3.1091433906555177

Epoch:    8/10    Loss: 2.960197114944458

Epoch:    8/10    Loss: 3.100936059951782

Epoch:    8/10    Loss: 2.9823217582702637

Epoch:    8/10    Loss: 2.931633024215698

Epoch:    8/10    Loss: 3.0497121238708496

Epoch:    8/10    Loss: 2.923986110687256

Epoch:    8/10    Loss: 3.0374154472351074

Epoch:    8/10    Loss: 3.2326363563537597

Epoch:    8/10    Loss: 3.1900796604156496

Epoch:    8/10    Loss: 2.968314628601074

Epoch:    8/10    Loss: 3.135648832321167

Epoch:    8/10    Loss: 3.009718084335327

Epoch:    8/10    Loss: 2.9291623878479003

Epoch:    8/10    Loss: 3.0833190727233886

Epoch:    8/10    Loss: 3.026272897720337

Epoch:    8/10    Loss: 2.808734483718872

Epoch:    8/10    Loss: 3.098991994857788

Epoch:    8/10    Loss: 3.051898899078369

E

Epoch:    9/10    Loss: 3.024244604110718

Epoch:    9/10    Loss: 3.0186448001861574

Epoch:    9/10    Loss: 2.8891765594482424

Epoch:    9/10    Loss: 2.932784299850464

Epoch:    9/10    Loss: 2.806267023086548

Epoch:    9/10    Loss: 2.9514266586303712

Epoch:    9/10    Loss: 2.975758123397827

Epoch:    9/10    Loss: 2.883191156387329

Epoch:    9/10    Loss: 2.774852085113525

Epoch:    9/10    Loss: 2.779380912780762

Epoch:    9/10    Loss: 2.8692523860931396

Epoch:    9/10    Loss: 2.8098456954956053

Epoch:    9/10    Loss: 2.76672420501709

Epoch:    9/10    Loss: 2.832266149520874

Epoch:    9/10    Loss: 2.7802035140991213

Epoch:    9/10    Loss: 2.7262729740142824

Epoch:    9/10    Loss: 2.887703685760498

Epoch:    9/10    Loss: 2.772450580596924

Epoch:    9/10    Loss: 2.9110805130004884

Epoch:    9/10    Loss: 2.757266626358032

Epoch:    9/10    Loss: 3.018036880493164

Epoch:    9/10    Loss: 2.8518111515045166

Epoch:    9/10    Loss: 3.028685703277588

Epo

Epoch:   10/10    Loss: 2.72979229927063

Epoch:   10/10    Loss: 2.7914614295959472

Epoch:   10/10    Loss: 2.8475175857543946

Epoch:   10/10    Loss: 2.6859889698028563

Epoch:   10/10    Loss: 2.7396948528289795

Epoch:   10/10    Loss: 2.883331546783447

Epoch:   10/10    Loss: 2.8484784507751466

Epoch:   10/10    Loss: 3.120259780883789

Epoch:   10/10    Loss: 2.9111406326293947

Epoch:   10/10    Loss: 2.814735326766968

Epoch:   10/10    Loss: 2.9161461448669432

Epoch:   10/10    Loss: 2.764593172073364

Epoch:   10/10    Loss: 2.9268598461151125

Epoch:   10/10    Loss: 2.8636585521697997

Epoch:   10/10    Loss: 2.913760166168213

Epoch:   10/10    Loss: 2.9139271450042723

Epoch:   10/10    Loss: 2.881564903259277

Epoch:   10/10    Loss: 2.851961431503296

Epoch:   10/10    Loss: 2.7876175117492674

Epoch:   10/10    Loss: 2.8117532920837403

Epoch:   10/10    Loss: 2.7334159660339354

Epoch:   10/10    Loss: 2.7453215503692627

Epoch:   10/10    Loss: 2.767174386978149

Epoch:   10/10    Loss: 2.8728575325012207

Epoch:   10/10    Loss: 2.8166943836212157

Epoch:   10/10    Loss: 2.8734183979034422

Epoch:   10/10    Loss: 3.007222671508789

Epoch:   10/10    Loss: 2.9504487037658693

Epoch:   10/10    Loss: 2.83138352394104

Epoch:   10/10    Loss: 2.9750442790985105

Epoch:   10/10    Loss: 2.8537125778198242

Epoch:   10/10    Loss: 2.7983189487457274

Epoch:   10/10    Loss: 2.8698663902282715

Epoch:   10/10    Loss: 2.8695687198638917

Epoch:   10/10    Loss: 2.6650820636749266

Epoch:   10/10    Loss: 2.8630099201202395

Epoch:   10/10    Loss: 2.9090867137908933

Epoch:   10/10    Loss: 2.969241352081299

Epoch:   10/10    Loss: 2.8614719200134275

Epoch:   10/10    Loss: 2.8409281635284422

Epoch:   10/10    Loss: 2.844747018814087

Epoch:   10/10    Loss: 2.838493423461914

Epoch:   10/10    Loss: 2.828055167198181

Epoch:   10/10    Loss: 2.916692018508911

Epoch:   10/10    Loss: 2.8808339977264406

Epoch:   10/10    Loss: 2.75593462944030


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** 
It is observed that a larger sequence length helps to back propogate through longer time frame, and hence the networks ability develop long term memory improves. This helps in better losses, but the memory and time complexity increases with increase in this parameters. So as a trade off, 32 is selected. Another factor is, if the sequence length is a power of 2, the internal prcessor computations would be much faster. A gradual increment from 8 to 64, is made, and its found that beyond 64 it takes lot of time to complete the training.

Large hidden dimentions will help the network to approximate complicated functions. Same logic as number of hidden layers in a FC network. However the backpropogation length increases, casusing extra memory and time ovrheads. A hidden dimention size of 1024 is decided, to get a trade off. Again , the parameter being power of 2 helps to improve processor coputation complexity

Started with a learning rate of 0.0001, and tried to gradually improved the same. However, the larger learning rate tends oscillate the loss. Lower learning rates are almost not reducing the loss. So its found 0.001 is the best initial value. Adams would adaptively modify this rate internally.

Number of epochs are selectd after observing  the validation, and training losses. The code for that is developed in PYCHARM IDE, and selected the point where the differences are small, and then both of them saturates. Currently it is set as 10, to achieve the required loss. However, its observed that higher number of epoches further reduces he loss, but will take hours to complete. 10 seems to be a better trade-off between the time and ojective in hand 

Number of layers are kept at 2, adding beyond that didnt seems to have any value addition to the performance

Ideal choise of batch size may be 1, meaning we need to do a fw and bw propogation and adjust weight for evry single data. This is practically not possible. If we increase the batch size to infinity, the processr will have to deal with an infinite sized matrix dimentions. Furthermore, the accuracy of weight adjustments will be deteriorated. So as a tradeoff, a batch size of 128 is selected. 
Generally embedding dimentions are chosen between 200 to 300. Beyond that it would not give any significant improvement

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

C:\Users\markkass\Anaconda3\envs\spyderRnn\lib\site-packages\torch\serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\markkass\Anaconda3\envs\spyderRnn\lib\site-packages\torch\serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\markkass\Anaconda3\envs\spyderRnn\lib\site-packages\torch\serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original sou

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [25]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
         current_seq = torch.LongTensor(current_seq)
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 32)

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [23]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [38]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.